# Dataset

First, we need to calculate a series of files that will ease our work before creating the dataset for training.
For most of recommendation systems problems. There is a two-step process to recommend something: Retrieval and Ranking. Howe

In [1]:
from pyspark.sql import SparkSession

sc = SparkSession.builder \
                 .appName("h&m_recommendations") \
                 .config("spark.sql.files.maxPartitionBytes", 5000000) \
                 .config("spark.sql.execution.arrow.pyspark.enabled", "true") \
                 .getOrCreate()
#spark = SparkSession(sc)


# Load transactions
transactions = sc.read.option("header", True) \
              .csv("./inputs/transactions_train.csv")

# Load articles
articles = sc.read.option("header", True) \
              .csv("./inputs/articles.csv")

# Load customers
customers = sc.read.option("header", True) \
              .csv("./inputs/customers.csv")

In [2]:
from pyspark.sql.functions import col

full_transactions = transactions.join(customers, on='customer_id', how='left')
full_transactions = full_transactions.join(articles, on='article_id', how='left')
full_transactions \
    .select(col('customer_id'), col('article_id'), col('age'), col('price'), col('prod_name'), col('product_type_name')) \
    .repartition(col('customer_id')) \
    .sortWithinPartitions(col('customer_id'), col('t_dat')) \
    .write \
    .partitionBy('customer_id') \
    .mode('overwrite') \
    .option('header', True) \
    .option('delimiter', ',') \
    .csv('./intermediate')



ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/Users/juanbeleno/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3437, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-2-cd06dd9a1a8a>", line 5, in <module>
    full_transactions \
  File "/Users/juanbeleno/opt/anaconda3/lib/python3.8/site-packages/pyspark/sql/readwriter.py", line 1240, in csv
    self._jwrite.csv(path)
  File "/Users/juanbeleno/opt/anaconda3/lib/python3.8/site-packages/py4j/java_gateway.py", line 1321, in __call__
    return_value = get_return_value(
  File "/Users/juanbeleno/opt/anaconda3/lib/python3.8/site-packages/pyspark/sql/utils.py", line 190, in deco
    return f(*a, **kw)
  File "/Users/juanbeleno/opt/anaconda3/lib/python3.8/site-packages/py4j/protocol.py", line 326, in get_return_value
    raise Py4JJavaError(
py4j.protocol.Py4JJavaError: <unprintable Py4JJavaError object>

During handling of the ab

ConnectionRefusedError: [Errno 61] Connection refused

ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/Users/juanbeleno/opt/anaconda3/lib/python3.8/site-packages/py4j/clientserver.py", line 516, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/juanbeleno/opt/anaconda3/lib/python3.8/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/Users/juanbeleno/opt/anaconda3/lib/python3.8/site-packages/py4j/clientserver.py", line 539, in send_command
    raise Py4JNetworkError(
py4j.protocol.Py4JNetworkError: Error while sending or receiving
